In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import joblib

df_binary = pd.read_csv('clean_data.csv')
df_categories = pd.read_csv('data.csv')

df_binary = df_binary.dropna(subset=['text']) #clean dataset
df_categories['severity_rating'] = [1 if n > 1 else n for n in df_categories['severity_rating']]
df_categories.rename(columns={'severity_rating': 'is_offensive'}, inplace=True)

# Combine the two datasets
df_combined = pd.concat([df_binary, df_categories], axis=0)

# Split the data for binary classification
X_train, X_test, y_train, y_test = train_test_split(df_combined['text'], df_combined['is_offensive'], test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = CountVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)

# Predict on the test set
y_pred = clf.predict(X_test_vec)

print(classification_report(y_test, y_pred))

# Split the data for category classification
X_train, X_test, y_train, y_test = train_test_split(df_categories['text'], df_categories['category_1'], test_size=0.2, random_state=42)

# Use the same vectorizer for category classification
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train the Model for category classification
category_clf = MultinomialNB()
category_clf.fit(X_train_vec, y_train)


#Generate dump of both ML models
joblib.dump(clf, 'profanityFilter.pkl')
joblib.dump(category_clf, 'profanityType.pkl')

              precision    recall  f1-score   support

         0.0       0.96      0.96      0.96     29502
         1.0       0.85      0.83      0.84      7688

    accuracy                           0.93     37190
   macro avg       0.90      0.89      0.90     37190
weighted avg       0.93      0.93      0.93     37190



['profanityType.pkl']

In [13]:
import joblib

#load models
clf = joblib.load('profanityFilter.pkl')
category_clf = joblib.load('profanityType.pkl')


################### ENTER PHRASE BELOW ###############################

userPhrase = ""

################### ENTER PHRASE ABOVE ###############################

# Test the model
phrase = [userPhrase]
phrase_vec = vectorizer.transform(phrase)

# Predict if the phrase is offensive
is_offensive = clf.predict(phrase_vec)

# Determine the category
if is_offensive[0] == 1:
    category = category_clf.predict(phrase_vec)
    print(f"The phrase is offensive and belongs to category [{category[0]}]")
else:
    print("The phrase is not offensive.")

<class 'SyntaxError'>: invalid syntax (<ipython-input-13-75606cb27d90>, line 7)